In [1]:
# imports
import pandas as pd

In [3]:
# import custom libraries and scripts
# sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(".")
sys.path.append("..")

from defaults import *
from extract_dataframe import read_json
from extract_dataframe import TweetDfExtractor
from clean_tweets_dataframe import Clean_Tweets

In [ ]:
# read csv
tweets_df = pd.read_csv("../data/clean_data.csv")
tweets_df.head()

: 

In [2]:
cleaner = Clean_Tweets(tweets_df.copy())

NameError: name 'Clean_Tweets' is not defined

### Sentiment analysis

: 

: 

### Topic modeling